# Test di ipotesi a varianza nota

Le specifiche tecniche per la velocità di combustione di un propellente richiedono che deve essere di 50 cm/s. Sappiamo che la deviazione standard è di 2 cm/s. Si effettuano 24 misurazioni
51, 50.2, 49.5, 48.7, 50.2, 50.5, 49.6, 51.1, 50.6, 49.1, 53.1, 50.4, 49.3, 48.9, 50.3, 51.8, 51.3, 48.5, 49.3, 55.1, 53.1, 52.5, 55.1, 50.6
- Si può rigettare l’ipotesi nulla con un livello di significatività del 5%?
- E se invece si richiedesse l’1%?
- Calcolare infine il p-value.

Supponiamo che lo sperimentatore voglia impostare il test in modo che la reale velocità di combustione media differisca da 50 cm/s per al più 1 cm/s.<br> Si vuole inoltre che il test affermerà questo fatto (cioè rigetterà H0 : μ = 50) con una probabilità del 90% e un livello di significatività del 5%.
- Determinare la dimensione campionaria.

In [1]:
import numpy as np
from scipy.stats import norm, t, chi2, binom

In [2]:
# Test bilatero
# H0 -> mu = mu_0
# H1 -> mu != mu_0

x = np.array([51, 50.2, 49.5, 48.7, 50.2, 50.5, 49.6, 51.1, 50.6, 49.1, 53.1, 50.4, 
              49.3, 48.9, 50.3, 51.8, 51.3, 48.5,49.3, 55.1, 53.1, 52.5, 55.1, 50.6])
sig = 2
mu_0 = 50
alpha = 0.05

n = x.size
mu = np.mean(x)
print(f'Media campionaria: {mu}')

# PAG 12
Z0 = (mu-mu_0)/sig * np.sqrt(n)
print(f'Z0: {Z0}')

# Test con livello di significativià al 5%
PHI = norm.ppf(1-alpha/2)
print(f'Phi: {PHI}')

cond = np.abs(Z0) > PHI
print("Rigetto l'ipotesi nulla" if cond else "Non ho elementi sufficienti per rigettare l'ipotesi nulla")

# poichè|Z0| > phi si rigetta l'ipotesi nulla
# quindi si afferma che il propellente non rispetta le specifiche con una significatività del 5%

Media campionaria: 50.824999999999996
Z0: 2.020829037796111
Phi: 1.959963984540054
Rigetto l'ipotesi nulla


In [3]:
# Test con livello di significativià al 1%
print(f'Z0: {Z0}')

alpha = 0.01
PHI = norm.ppf(1-alpha/2)
print(f'Phi: {PHI}')

cond = np.abs(Z0) > PHI
print("Rigetto l'ipotesi nulla" if cond else "Non ho elementi sufficienti per rigettare l'ipotesi nulla")

# poichè |Z0| < PHI non abbiamo sufficienti elementi per rigettare l'ipotesi nulla

Z0: 2.020829037796111
Phi: 2.5758293035489004
Non ho elementi sufficienti per rigettare l'ipotesi nulla


In [4]:
# p-value : il più piccolo valore di alpha per il quale è possibile rigettare l'ipotesi

# Pag 14
p = 2 * (1 - norm.cdf(np.abs(Z0)))
print(f'P-value: {round(p, 2)}\n')

P-value: 0.04



In [5]:
# Calcoliamo la dimensione campionaria tramite errore di secondo tipo

alpha = 0.05
beta = 1 - 90/100 # probabilità di non rifiutare H0 quando è falsa
delta = 1 # quanto ci stacchiamo dalla media vera

PHI_alpha = norm.ppf(1-alpha/2)
print(f'Phi: {PHI_alpha}')

# Pag 16
check = -PHI_alpha-delta*np.sqrt(n)/sig # se <= -3 approssima una normale
print(f'Vale {check} <= -3, quindi la distribuzione approssima una normale') 

PHI_beta = norm.ppf(1-beta)
dim_camp = (PHI_alpha+PHI_beta)**2 * (sig**2/delta**2)
print(f"Dimensione del campione: {int(dim_camp)}")

Phi: 1.959963984540054
Vale -4.409453727323232 <= -3, quindi la distribuzione approssima una normale
Dimensione del campione: 42


# Test sulla media con varianza ignota

Si vuole testare ad un livello di significatività α = 0.05 se il carico di rottura di un materiale supera 10 MPa, tenendo presente che 22 prove hanno fornito i seguenti risultati
19.8 18.5 17.6 16.7 15.8 15.4 14.1 13.6 11.9 11.4 11.4 8.8 7.5 15.4 15.4 19.5 14.9 12.7 11.9 11.4 10.1 7.9
- Calcolare inoltre il p-value.

In [6]:
# Test unilatero a destra
# Ipotesi nulla         H_0 : mu = mu_0
# Ipotesi alternativa   H_1 : mu > mu_0

x = np.array([19.8, 18.5, 17.6, 16.7, 15.8, 15.4, 14.1, 13.6, 11.9, 11.4, 11.4,
              8.8, 7.5, 15.4, 15.4, 19.5, 14.9, 12.7, 11.9, 11.4, 10.1, 7.9,])

alpha = 0.05
mu_0 = 10

n = x.size
mu = np.mean(x)
print(f'Media campionaria: {mu}')
S = np.std(x, ddof=1)
print(f'Deviazione standard campionaria: {S}')

# Pag 19
T0 = (mu-mu_0)/S * np.sqrt(n)
print(f'T0: {T0}')

# Test unilatero a destra
T = t.ppf(1-alpha, n-1)
print(f't: {T}')

cond = T0 > T
print("Rigetto l'ipotesi nulla" if cond else "Non ho elementi sufficienti per rigettare l'ipotesi nulla")
    
# T0 è maggiore di t pertanto si rigetta l'ipotesi nulla in favore dell'ipotesi alternativa
# affermando che il carico di rottura supera significativamente il valore di 10

Ft = t.cdf(T0, n-1)
p = 1-Ft
print(f'P-value: {p}')

Media campionaria: 13.713636363636363
Deviazione standard campionaria: 3.5535757249424744
T0: 4.901682101212391
t: 1.7207429028118775
Rigetto l'ipotesi nulla
P-value: 3.781272593450513e-05


# Test sulla varianza

Un macchinario riempie automaticamente delle bottiglie. Da un campione di 20 misurazioni si ottengono i seguenti valori (in litri)
2.05, 2.04, 1.98, 1.96, 2.03, 2.01, 1.97, 1.99, 2.01, 2.05 1.96, 1.95, 2.04, 2.01, 1.97, 1.96, 2.02, 2.04, 1.98, 1.94
Se la deviazione standard fosse superiore a 0.05 litri, la proporzione di bottiglie sotto o sovrariempite sarebbe non accettabile.

- I dati del campione contengono prove che suggeriscono che il produttore abbia un problema con le bottiglie riempite troppo o troppo poco?<br>Utilizzare α = 0.05 e assumere che il volume di riempimento abbia una distribuzione normale.

In [7]:
# Test unilatero a sinistra
# HP0: sig^2 = sig0^2
# HP1: sig^2 < sig0^2 perchè voglio testare che sia accettabile (se fosse superiore non sarebbe accettabile)

x = np.array([2.05, 2.04, 1.98, 1.96, 2.03, 2.01, 1.97, 1.99, 2.01, 2.05, 1.96, 1.95, 2.04, 2.01, 1.97, 1.96, 2.02, 2.04, 1.98, 1.94])
n = x.size
sig0 = 0.05
alpha = 0.05

S = np.std(x, ddof=1)
print(f'Deviazione standard campionaria: {S}')

# Pag 24
W0 = S**2 / sig0**2 * (n-1)
print(f'W0: {W0}')

CHI = chi2.ppf(alpha, n-1)
print(f'CHI: {CHI}')

cond = W0 < CHI 
print("Rigetto l'ipotesi nulla" if cond else "Non ho elementi sufficienti per rigettare l'ipotesi nulla")
# Dato che W0 < CHI posso rigettare l'ipotesti nulla a favore dell'ipotesti alternativa, quindi la produzione sta andando bene

Deviazione standard campionaria: 0.03621572790057408
W0: 9.967999999999988
CHI: 10.117013063859044
Rigetto l'ipotesi nulla


# Test sulla proporzione
In una catena di produzione si vuole mantenere il numero di pezzi difettosi al di sotto del 5%. Si analizza un campione di 200 pezzi e si trovano 4 pezzi difettosi.
- Si può asserire ad un livello di significativita α = 0.05 che la produzione rispetta le aspettative?
- Supponendo che il valore vero sia p∗ = 0.03 e supponendo che il costruttore voglia accettare un valore dell’errore di secondo tipo β = 0.1, quale ampiezza dovrebbe avere il campione?

In [8]:
# Test unilatero a sinistra
# HP0 = p = p0
# HP1 = p < p0

p0 = 5/100
n = 200
k = 4   # numero di successi --> numero di pezzi difettosi
alpha = 0.05

# Pag 26
p = k/n  # media campionaria
print(f'Media campionaria: {p}')

Z0 = (p-p0) / (np.sqrt( p0*(1-p0) )) * np.sqrt(n)
print(f'Z0: {Z0}')

PHI_alpha = norm.ppf(alpha)
print(f'PHI: {PHI_alpha}')

cond = Z0 < PHI_alpha
print("Rigetto l'ipotesi nulla" if cond else "Non ho abbastanza elementi per rigettare l'ipotesi nulla")
# dato che Z0 < PHI si rigetta l'ipotesi nulla e si avalla l'ipotesi alternativa quindi le aspettative di produzione sono rispettate

Media campionaria: 0.02
Z0: -1.946657053569151
PHI: -1.6448536269514729
Rigetto l'ipotesi nulla


In [9]:
p_star = 0.03
beta = 0.1

PHI_beta = norm.ppf(beta)
print(f'PHI_beta: {PHI_beta}')

dim_campionaria = ((PHI_beta * np.sqrt(p_star*(1-p_star))) + (PHI_alpha * np.sqrt(p0*(1-p0))) / (p0-p_star) )**2
print(f'Dimensione campionaria: {int(dim_campionaria)}')

PHI_beta: -1.2815515655446004
Dimensione campionaria: 329


# Test sulla media per coppie di popolazioni

Si vogliono confrontare due tipi di preparati per pittura. Ci si aspetta un diverso tempo di essiccamento. Si può supporre che la deviazione standard del tempo di essiccamento per ciascun tipo di essiccamento sia 8 minuti. 10 pareti vengono tinteggiate con il trattamento 1 e altrettamente pareti con il trattamento 2. <br>Si rilevano le medie campionare X_ = 121 minuti e Y_ = 112 minuti. 
- Si può trarre la clusione che il tempo di essiccamento del campione 1 sia maggiore di quello del campione 2 assumendo α = 0.05?
- Calcolare l’intervallo di confidenza per la differenza dei tempi medi di essiccamento.

In [18]:
# Test unilatero a destra
# H0: mu1 = mu2 
# H1: mu1 > mu2 

sig1 = sig2 = 8   # Deviazione standard comune per entrambi i trattamenti
n = m = 10     

mu1 = 121  
mu2 = 112 

alpha = 0.05 

# Pag 31
sig2_1 = sig2_2 = sig1**2
Z0 = (mu1-mu2)/np.sqrt((sig2_1/n) + (sig2_2/m))
print(f'Z0: {Z0}')

PHI = norm.ppf(1-alpha)
print(f'PHI: {PHI}')

cond = Z0 > PHI
msg = "Si può trarre la clusione che il tempo di essiccamento del campione 1 sia maggiore di quello del campione 2 assumendo α = 0.05\n"
print(f"\nRigetto l'ipotesi nulla\n{msg}" if cond else "\nNon ho abbastanza elementi per rigettare l'ipotesi nulla\n")

# Se si vuole calolare il p-value
p_value = 1 - norm.cdf(Z0)
print(f'P-value: {p_value}')

# Calcolo dell'intervallo di confidenza per la differenza delle medie
A = "-inf"
B = mu1-mu2-PHI*np.sqrt((sig2_1/n) + (sig2_2/m))

msg = f'''Vale a dire che il tempo di essiccamento del primo preparato per pittura è di almeno {round(B, 2)} minuti
maggiore di quello del secondo preparato per pittura con una significatività del 95%.'''
print(f'\nIntervallo di confidenza: ]{A}, {round(B,2)}[\n{msg}')

Z0: 2.5155764746872635
PHI: 1.6448536269514722

Rigetto l'ipotesi nulla
Si può trarre la clusione che il tempo di essiccamento del campione 1 sia maggiore di quello del campione 2 assumendo α = 0.05

P-value: 0.00594189462107364

Intervallo di confidenza: ]-inf, 3.12[
Vale a dire che il tempo di essiccamento del primo preparato per pittura è di almeno 3.12 minuti
maggiore di quello del secondo preparato per pittura con una significatività del 95%.
